# Information Theory and the Game Wordle (Part 1 of 2)

## Goal of this lecture (Part 1 of 2):

This lecture will be split up into 2 parts. This first part will introduce the game wordle and the basic concepts of Information Theory that we will use. All the code written in this part will be in Julia. The 2nd part of this lecture will take our knowledge of Information Theory and combine it with the game Wordle to create an algorithm that would give us an optimal guess of words given the current information.

## What is Wordle?

If you don't already know what the game Wordle is, you might be living under a rock. However, if that is true, you don't have to worry. That is because Wordle is a simple word game that operates off of a couple of rules.

In this game, you'll have 6 guesses to find the hidden 5 letter word. For every letter of each guess, you will be told either:

1. The letter does not exist in the word, marked with a gray box
2. The letter exists in the word but is not in the right spot, marked with a yellow box
3. The letter exists in the word and is in the right spot, marked with a green box

**Here is an example:**
Our first guess was "CRATE". Since the letters "CRAT" are in gray boxes, we know that none of those letters exist in this word. However, the last E is in a green box! So we know we got one! It is both in the word and in the right location!

Our next guess is "DRIED".  Since the first D is now in a green box, we know the word begins with a D! Since there is another D in a yellow box, we know that there is at least another D somewhere in the word! But don't forget that we already know that the last letter must be an E since we got that information from the last guess! So, even though it is highlighted yellow, we already know where it goes!

Our final guess is the word "DODGE", which shows up in all green which means that we got the word in just 3 guesses!\
![Alt text](wordleExample.png)

## What is our goal?

The goal of the game Wordle is to find the correct word within 6 guesses. However, for most players, we enjoy finding the word in the least amount of guesses possible! So, the purpose of this project is to create an algorithm that could help us beat Wordle in the least amount of guesses possible! (On average)

## How can we achieve this?
One thing that we could try to do is to remove as many possibilities per guess as possible! The way that we would do this for a game like Wordle may not be immediately obvious. However, we can take inspiration from a simpler guessing game.

Instead of Wordle, imagine we are playing a game where we have to guess a number between 1 and 100. After each guess, we will be told whether or not our answer is too high or too low! 

## Example
Let's let the hidden answer be 100.

If you think about it, the least optimal strategy for this guessing game would be to guess numbers in order from 1 to 100. That is because, in this worst-case scenario, we will have to make 100 guesses to get the right answer!\
1 (low), 2 (low), 3 (low), 4 (low), ..., and so on until we reach 100!

Instead, the optimal strategy is to cut remaining possible guesses in half each time based on the information given to us! So, since our guessing range starts off at 1 to 100, we will guess right in the middle. Watch how this works!
1. 50 (low, so well go into the middle of the higher interval 51 - 100)
2. 75 (low, so well go into the middle of the higher interval 76 - 100)
3. 88 (low, so well go into the middle of the higher interval 89 - 100)
4. 94 (low, so well go into the middle of the higher interval 95 - 100)
5. 97 (low, so well go into the middle of the higher interval 98 - 100)
6. 99 (low, so well go into the middle of the higher interval 100 - 100)
7. 100 (Correct!)

(To clarify, since these intervals usually don't have an exact middle number, we favor the left middle of the interval. It would work just as fine if we favored the right side.)

Using this algorithm, we were able to reduce our number of guesses from 100 to just 7! In fact, using this algorithm, the highest amount of guesses required to find the answer would be $\log_2(n)$ where n is the number of possible guesses. So that would mean that most amount of guesses for our number game is just 7! Which is way better than 100.

This is actually the equivalent of a binary search on a list. Here is what the implementation for what we did above would look like:

In [6]:
# Recursive example of this binary search
function numberGuesser(rangeStart, rangeEnd, target)
    if rangeStart > rangeEnd
        return
    end
        
    currentGuess = floor((rangeStart + rangeEnd) / 2) # Sets currentGuess to left middle of the range
    println(currentGuess)
        
    if currentGuess == target
        return
    elseif currentGuess > target
        numberGuesser(rangeStart, currentGuess - 1, target)
    else
        numberGuesser(currentGuess + 1, rangeEnd, target)
    end
end

for i in 4:2:10
    println(string(i^2, ": "))
    numberGuesser(1, 100, 2*i^2)
    println()
end

16: 
50.0
25.0
37.0
31.0
34.0
32.0

36: 
50.0
75.0
62.0
68.0
71.0
73.0
72.0

64: 
50.0
75.0
88.0
94.0
97.0
99.0
100.0

100: 
50.0
75.0
88.0
94.0
97.0
99.0
100.0



The implementation above favors the left middle of the interval by rounding down using the floor function. So, just by replacing that single function call with the ceil function, we can make it favor the right side of the interval.

In [7]:
# Recursive example of this binary search
function numberGuesser(rangeStart, rangeEnd, target)
    if rangeStart > rangeEnd
        return
    end
        
    currentGuess = ceil((rangeStart + rangeEnd) / 2) # Sets currentGuess to right middle of the range
    println(currentGuess)
        
    if currentGuess == target
        return
    elseif currentGuess > target
        numberGuesser(rangeStart, currentGuess - 1, target)
    else
        numberGuesser(currentGuess + 1, rangeEnd, target)
    end
end

for i in 4:2:10
    println(string(i^2, ": "))
    numberGuesser(1, 100, 2*i^2)
    println()
end

16: 
51.0
26.0
39.0
33.0
30.0
32.0

36: 
51.0
76.0
64.0
70.0
73.0
72.0

64: 
51.0
76.0
89.0
95.0
98.0
100.0

100: 
51.0
76.0
89.0
95.0
98.0
100.0



Running this version of the implementation, we can actually see that it is able to get some numbers in 1 fewer guess. However, on average, they will get numbers on the same amount of guesses.

So, this is an easy example to show what we want to achieve with Wordle. Which is great and all. How would we even achieve this in Wordle? Words aren't ordered and we aren't told if a word is too high or too low!

The way that we approach Wordle is to guess a word that we hope would give us clues onto what the actual word might be. However, as we mentioned before, the ideal approach would be to guess words that would eliminate the most possible words each time! But humans are incapable of doing that in their own brains!

Here is where Information Theory will come in handy!

## Basics of Information Theory

In Information Theory, the unit of information used is called **the bit**. The amount of bits is usually represented with the letter I (for information) and can be solved with this equation $I = -\log_2(p)$ where p is how much smaller the space of possibilities becomes. Here is an example:

In the number guessing game, since each of the guesses that we made using the binary search method split the amount of possibilities in half, the amount of information we would get is $I = -\log_2(1/2) = 1$! Because of this, we can see that 1 bit of information means that we have lowered the remaining possibilities in half! So, each subsequent guess using the binary search method will give us about 1 bit of information!

Now, here is an example with Wordle!

## Wordle example!

In [8]:
# This Julia file includes all the words that Wordle will accept as a guess, as well as all the possible answers.
include("words.jl") # I stole this information straight from the Wordle source code. 

length(words) + length(answers) # The words and answers are disjoint, so we must combine them to find the total number of possible guesses!

12972

Running the code above shows sus that there are 12972 possible guesses that Wordle will accept! In reality, there are less words that it would use as answers, but let's ignore that for now. 

Now, let's say that the word CCRATE cuts the number of guesses down from 12972 to just 810, then the amount of guesses have been cut down by a factor of almost 16! That means that amount of information that the word CRATE gives us is about $I = -\log_2(1/16) = 4$! By finding the word for each guess that gives us the most amount of information or bits, we can reduce the number of possible guesses by as much as possible per guess!

In [10]:
-log(810/12972)/log(2) # Number of bits of information from our example

4.00133521068382

So, now that we know that information is measured in bits, and the more bits that a word gives, the more it reduces the possibilities, how can we find which word would give us the most amount of information?

## Entropy!

There is a way for us to calculate the expected information that a word could give us! This function looks like this $E(w) = \displaystyle\sum_{x}-p(x)\log_2(p(x))$. This is called the **Entropy Function**.\
Here, $w$ is the word that we are guessing. $x$ is one of the possible results that could appear on the screen, and $p(x)$ is the probability of each of those results.

For example, if the word w is "CRATE", the first x could be the result where all letters are gray. Then, $p(x)$ is the probability of getting that result, and $-log_2(x)$ is the amount of information of that result. When we take the sum this equation with all possible results for the word w, we get the expected information of each guess. Don't worry if this is confusing now, because we will implement this in part 2 using Python.


## What is the most information we could obtain using this?
First, we need to assume there is a uniform distribution for each result. That means that getting all grays, all yellows, all greens,and anything in between is just as likely. Then, the Entropy function would be equal to $\log_2(3^5)$ since there are $3^5$ equally likely outcomes.

In [11]:
log(3^5)/log(2)

7.924812503605781

The result of that is about 8. Which means that, at most, we could expect that a word would be able to cut our set down in half about 8 times! So there is a strict upperbound for our expected entropy value. However, since the probability of each outcome is not uniform, we can expect our entropy values to always be lower than that for all of our guesses.

## Conclusion for part 1!

Now that you understand how the game Wordle works, and the basics of Information Theory, we can now write a program that can find us the optimal guess! We will now go to part 2 to implement these ideas in Python!